# Ejemplo Multiplicación matrices


En este ejemplo se realiza la maultiplicación de 2 matrices cuadradas usando GPU

---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=661775 sha256=3d8c4259a84f8c82812243268c180b0b2c137283753f80687b0c1cf0a6444a24
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=e3340464c1394993b9e86bc05bd8f271d786b93cbb13925795d7139fa0e66337
  Stored in directory: /root/.cache/pip/wheels/cb/fc/a

---
# 3 Desarrollo
Ejecución del algoritmo de multiplicación de matrices en GPU.

In [ ]:
#%%writefile matriz_mul.py
#Este código define dos matrices cuadaras  `a` y `b . Luego alloca memoria en la GPU para cada una de ellas y copia los datos a la GPU. 
#Luego, define una función de kernel de CUDA llamada `matrix_mul` que realiza la multiplicación de matrices. La función de kernel
# se llama a través de PyCUDA y se proporciona con los datos de entrada y salida y la configuración de bloques y grupos de hilos.
#Finalmente, se copia el resultado de la GPU de nuevo al host y se imprime el resultado.

import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import math
import pdb

# Define the kernel function
from pycuda.compiler import SourceModule
from datetime import datetime


# --------------------------------------------
# Formulario que defiene la cantidad de filas y columnas que van a tener la matriz A y B
#@markdown Ingrese las filas columnas para las matrices A y B. A es (mxn) y B es (nxp)
m = 2 #@param {type:"number"}
n = 2 #@param {type:"number"}
p = 2 #@param {type:"number"}
# --------------------------------------------

# -----------------------------------------------------------------------------

def Mostrar(matriz):
    for fila in matriz:
        for valor in fila:
            print("\t", valor, end=" ")
        print()

def main():
    
    
    # Inicializar matrices A y B con numeros aleatorios
    a = np.random.randint(10, size=(n, m))
    b = np.random.randint(10, size=(m, p))
    a =np.int32(a)
    b =np.int32(b)

    # Obtenga la dimensión de las matrices
    cant_a_row = a.shape[0]
    cant_a_col = a.shape[1]
    cant_b_row = b.shape[0]
    cant_b_col = b.shape[1]

    total_elements_matrix_c=cant_a_row*cant_b_col
    
    #Se verifica que se pueda multiplicar la matriz
    if cant_a_col!=cant_b_row:
      print("No se puede multiplicar las matrices, ya que no coincide el N° de columnas de A con el N° de filas de B")
      return False
   
    
    #creo la matriz resultante, llena con ceros
    c = np.zeros((cant_a_row,cant_b_col)).astype(np.int32)
    
    # Allocate memory on the GPU
    a_gpu = cuda.mem_alloc(a.nbytes)
    b_gpu = cuda.mem_alloc(b.nbytes)
    c_gpu = cuda.mem_alloc(c.nbytes)

    # Copy the data to the GPU
    cuda.memcpy_htod(a_gpu, a)
    cuda.memcpy_htod(b_gpu, b)
    cuda.memcpy_htod(c_gpu, c)

    mod = SourceModule("""
      __global__ void matrix_mul(int *c, int *a, int *b, int num_row_a, int num_col_a,int num_row_b, int num_col_b)
      {
          int row = blockIdx.y*blockDim.y+threadIdx.y;
          int col = blockIdx.x*blockDim.x+threadIdx.x;
          

          //Como se trabaja con 2 matrices y el resultado C (N*P) es la fusion de A(N*M) y B(M*P) 
          //los limites del if se comprueban con los limites de estos dos.
          //row con num_row_a por (N) y col con num_col_b por (P)
          if(row<num_row_a && col <num_col_b) 
          {
            float value = 0;

            for(int k=0;k<num_col_a;k++) 
              //Esta es la configuracion para hacer multiplicación de matrices cuadraticas
              value += a[row*num_col_a+k]*b[k*num_col_a+col];       
              
            c[row*num_col_b+col] = value;

         }
        }
      """)
    
    func = mod.get_function("matrix_mul")

    # Define el tamaño de los bloques y los grupos de hilos
    threads_per_block = (8,8,1)
    
    cant_blocks_x = (math.ceil(cant_b_col / threads_per_block[0]))
    cant_blocks_y = (math.ceil(cant_a_row / threads_per_block[1]))
    blocks_per_grid = (cant_blocks_x,cant_blocks_y ,1)
 
    func(c_gpu, a_gpu, b_gpu, np.int32(cant_a_row), np.int32(cant_a_col),np.int32(cant_b_row), np.int32(cant_b_col), block=threads_per_block, grid=blocks_per_grid)

    # Se copia los resultados de la memoria de la GPU a la CPU
    cuda.memcpy_dtoh(c, c_gpu)
    

    # Se imprimen los resultados 
    print("--------------------------------------------")    
    print("Elementos matriz c: ",       total_elements_matrix_c)
    print("Thread por bloques: ",       threads_per_block)
    print ("Bloques por grid (grilla): ",blocks_per_grid)

    print("La matriz A es la siguiente:")
    Mostrar(a)

    print("\nLa matriz B es la siguiente:")
    Mostrar(b)
    
    print("\nLa matriz C es la siguiente:")
    Mostrar(c)
    
    # Se libera la memoria de la GPU
    a_gpu.free()
    b_gpu.free()
    c_gpu.free()
    
    return True

if __name__ == "__main__":
    resp=main()


--------------------------------------------
Elementos matriz c:  4
Thread por bloques:  (8, 8, 1)
Bloques por grid (grilla):  (1, 1, 1)
La matriz A es la siguiente:
	 4 	 2 
	 9 	 4 

La matriz B es la siguiente:
	 2 	 1 
	 0 	 1 

La matriz C es la siguiente:
	 8 	 6 
	 18 	 13 


## 3.1 Ejecución y profiling del script 

Uitlizando nvprof se ejecuta el script y se miden los tiempos de ejecución.

In [ ]:
!nvprof python matriz_mul.py

==3429== NVPROF is profiling process 3429, command: python3 matriz_mul.py
--------------------------------------------
Elementos matriz c:  4
Thread por bloques:  (8, 8, 1)
Bloques por grid (grilla):  (1, 1, 1)
La matriz A es la siguiente:
	 6 	 5 
	 0 	 1 

La matriz B es la siguiente:
	 1 	 5 
	 1 	 7 

La matriz C es la siguiente:
	 11 	 65 
	 1 	 7 
==3429== Profiling application: python3 matriz_mul.py
==3429== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.81%  5.6000us         1  5.6000us  5.6000us  5.6000us  matrix_mul
                   37.17%  4.5440us         3  1.5140us  1.3760us  1.7920us  [CUDA memcpy HtoD]
                   17.02%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   82.64%  287.08ms         1  287.08ms  287.08ms  287.08ms  cuCtxCreate
                   17.14%  59.549ms         1  59.549ms  59.549ms  59.549ms  cuCtxDetach
                    0.0

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)


